# Задание 6

### Пример входных данных

In [1]:
import pandas as pd
clinton = pd.read_csv("clinton.csv", encoding="utf-8")
trump = pd.read_csv("trump.csv", encoding="utf-8")

In [2]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [3]:
from random import randint

In [4]:
from collections import Generator

class Agent(Generator):
    
    def __init__(self, kb, name):
        # Инициализация
        self.tweets_pool = kb['text']
        self.num_tweets = len(kb)
        self.name = name

    def send(self, msg):
        # Будем возвращать случайное сообщение из self.tweets_pool
        random_idx = randint(0, self.num_tweets - 1)
        return "{} : {}".format(self.name, self.tweets_pool[random_idx])

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def __str__(self):
        # Опишем строковое представление класса
        return self.name

    def __repr__(self):
        return str(self)

#### Пример использования:

In [3]:
from dialogsgenerator import Agent
clint, trum = Agent(clinton, "clinton"), Agent(trump, "trump")
clint.send("bla bla")

'clinton : One candidate made it clear he wasn’t prepared for last night’s debate. The other made it clear she’s prepared to b… https://t.co/InYZBmnbBM'

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [7]:
import sys
from itertools import chain, repeat

In [33]:
class RandomDialog(object):
    
    def __init__(self, agents, max_len=5):
        # Инициализация
        self.agents = agents
        self.agents_num = len(agents)
        self.max_len = max_len
    
    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
        
        """
        ______________________________________________________________________________________
            Мы хотим вернуть (max_len - 1) цепочек ходов, для этого нам нужно (max_len - 1) раз 
            вызвать turn(), но чтобы каждый раз метод turn() возвращал разные диалоги.

             * self_n_times - пары (i, self), где i = 0..max_len-1
             * invoke_turn - вызывает функцию turn у своего второго аргумента (второй аргумент - self)

            Возьмём map от self_n_times, к котором применён invoke_turn, получим то, что нужно            
        """
        
        # пары - (i, self), где i = 1..max_len-1
        self_n_times = zip(range(self.max_len - 1), repeat(self))
        
        # вызывает turn() у второго аргумента
        invoke_turn = lambda args: list(args[1].turn())
        
        # для каждой пары (i, self) вызываем invoke_turn от self
        yield from map(invoke_turn, self_n_times)

    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """
        
        # получаем сообщение от случайного агента
        rand_agent_ind = randint(0, self.agents_num - 1)
        msg = next(self.agents[rand_agent_ind])
        
        # отправляем сообщение всем агентам
        _ = list(map(lambda x: x.send(msg), self.agents))

        # возвращаем ответы всех агентов вместе с исходным сообщением
        it = iter(self.agents)
        yield from chain([msg], list(map(next, it)))

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
                
        return list(dialog)      

    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        
        if dialog is None:
            dialog = self.eval()
        
        # приписывает к очередному ходу \t
        sep_turns = lambda turn: "\t" + turn
        
        # описание диалога - сначала отдаёт строку с номером диалога, а затем сами сообщения
        turn_desc = lambda turn_obj: list(chain(["turn {}:".format(turn_obj[0])], 
                                           map(sep_turns, turn_obj[1])))
        
        # выводит каждое описание диалога
        print_dialog = lambda turn: list(map(lambda x: print(x, file=target), turn_desc(turn)))
        
        list(map(print_dialog, enumerate(dialog)))
        
    pass


In [4]:
from dialogsgenerator import RandomDialog
test_random_diaog = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=3)

In [5]:
gen = test_random_diaog.turn()
list(gen)

['trump : Nasty Ted Cruz is at it again- same dirty tricks he used w/ @RealBenCarson- saying I may not be on ballot &amp; I hold liberal positions. LIES!',
 'clinton : “Donald just criticized me for preparing for this debate. And you know what else I prepared for?\n\nI prepared to be… https://t.co/QjB01msPaT',
 'trump : Thank you @RepReneeEllmers!\nhttps://t.co/1DrZpfa6je https://t.co/rzEw8pQhds']

In [6]:
gen_dialog = test_random_diaog.generate()
list(test_random_diaog.generate())
gen_dialog

<generator object RandomDialog.generate at 0x000002825F9E05C8>

In [7]:
evaluate = test_random_diaog.eval(gen_dialog)

In [8]:
test_random_diaog.write(evaluate)

turn 0:
	trump : MAKE AMERICA GREAT AGAIN!
	clinton : 62 reasons why Donald Trump should never become president (from members of his own party). https://t.co/0gYEctRjSe
	trump : In interview I told @AP that my taxes are under routine audit and I would release my tax returns when audit is complete, not after election!
turn 1:
	trump : "@redneckgp: All you haters out there, STOP trashing the only candidate @realDonaldTrump that will put ALL OF YOU &amp; AMERICA FIRST  #trump"
	clinton : West Virginians, head to the polls today. RT this if you’re voting for Hillary. https://t.co/2ECEOO7DYh
	trump : Crooked Hillary Clinton knew everything that her "servant" was doing at the DNC - they just got caught, that's all! They laughed at Bernie.


#### Пример использования:

In [17]:
from dialogsgenerator import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

In [10]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x000002E9CB2297D8>

In [11]:
evaled = rd.eval(generated)
evaled

[['clinton : "We have a chance to show that comprehensive immigration reform isn’t just the smart thing to do—it’s what the American people demand."',
  "clinton : Trump asked what the African American community has to lose by voting for him. @MarlonDMarshall's response: https://t.co/ZJtX7d49bM",
  'trump : Get out and VOTE tomorrow! We will MAKE AMERICA GREAT AGAIN! \n#CTPrimary #DEPrimary #MDPrimary #PAPrimary #RIPrimary\nhttps://t.co/ndi8sZI1Gt']]

In [12]:
rd.write(evaled)

turn 0:
	clinton : "We have a chance to show that comprehensive immigration reform isn’t just the smart thing to do—it’s what the American people demand."
	clinton : Trump asked what the African American community has to lose by voting for him. @MarlonDMarshall's response: https://t.co/ZJtX7d49bM
	trump : Get out and VOTE tomorrow! We will MAKE AMERICA GREAT AGAIN! 
#CTPrimary #DEPrimary #MDPrimary #PAPrimary #RIPrimary
https://t.co/ndi8sZI1Gt


### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="cmd.png">
Описание аргументов представлено на рисунке:
<img src="cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [62]:
def generate(rd, count_dialogs=5):
    """
    Генерирует count_dialogs диалогов с помощью rd.generate().
    Возвращаемый объект является генератором.
    """
    # пары - (i, rd), где i = 1..count_dialogs
    rd_n_times = zip(range(count_dialogs), repeat(rd))

    # вызывает generate() у второго аргумента
    invoke_generate = lambda args: args[1].generate()

    # у каждой пары (i, rd) вызываем invoke_generate от rd
    yield from map(invoke_generate, rd_n_times)
    

def write(dialogs, target):
    """
    Записывает сгенерированные диалоги dialogs (это объект-генератор) в поток target.
    """

    # выводит сначала номер диалога, а затем вызывает функцию write у самого диалога
    def print_dialog_descr(dialog_obj):
        print("_______________________Dialog {}______________________".format(dialog_obj[0]), file=target)

        rd = RandomDialog([], 0)
        evaluate = rd.eval(dialog_obj[1])
        rd.write(evaluate)
        
    
    print_dialogs = lambda dialog_obj: print_dialog_descr(dialog_obj)
   
    list(map(print_dialogs, enumerate(dialogs)))


if __name__ == "__main__":
    print("sdf")


sdf


In [6]:
import sys
from run_generator import generate, write

In [7]:
dialogs = generate(test_random_diaog, 2)
dialogs

<generator object generate at 0x00000284C4B9B620>

In [8]:
write(dialogs, sys.stdout)

_______________________Dialog 0______________________
turn 0:
	clinton : La próxima semana, los republicanos nominarán a alguien que piensa que una campaña para la comunidad latina es tuitear un 'taco bowl'.
	clinton : “I’m going after him exactly on those issues and statements that are divisive and dangerous.” —Hillary on @realDonaldTrump
	trump : I don't believe I have been given any credit by the voters for self-funding my campaign, the only one. I will keep doing, but not worth it!
turn 1:
	trump : The polls show that I picked up many Jeb Bush supporters. That is how I got to 46%. When others drop out, I will pick up more.  Sad but true
	clinton : Watch live as Hillary stops in St. Petersburg, Florida, for a campaign rally on the choice we face on the economy: https://t.co/09cW39nHXo
	trump : I don’t cheat at golf but @SamuelLJackson cheats—with his game he has no choice—and stop doing commercials!
_______________________Dialog 1______________________
turn 0:
	clinton : Shirley was

#### Пример использования функций из файла:

In [18]:
import sys
from run_generator import generate, write

In [19]:
dialogs = generate(rd, 2)
dialogs

<generator object generate at 0x00000284C4C02620>

In [20]:
write(dialogs, sys.stdout)

_______________________Dialog 0______________________
turn 0:
	trump : Going to Salt Lake City, Utah, for a big rally. Lyin' Ted Cruz should not be allowed to win there - Mormons don't like LIARS! I beat Hillary
	clinton : Cheryl's husband died serving our country. 

Trump's company scammed her out of money she needed for her family. https://t.co/gfMeiJZ3DW
	trump : Why do the networks continue to put dopey @BillKristol on panels when he has called every single shot about me wrong for 2 yrs?
_______________________Dialog 1______________________
turn 0:
	clinton : "We respect the choice the people of the United Kingdom have made." —Hillary #BrexitVote https://t.co/58yM8oDkL6
	clinton : 22 memorable moments from the Democratic Convention that you probably missed: https://t.co/xnvFiEKlJC https://t.co/bLqoDfFfvc
	trump : The invention of email has proven to be a very bad thing for Crooked Hillary in that it has proven her to be both incompetent and a liar!
